## **Database Connectivity in DataRobot**

As always, reference the Python package `datarobot` api docs: https://datarobot-public-api-client.readthedocs-hosted.com/en/v2.21.3/setup/getting_started.html

Databases are a widely used tool for carrying valuable business data. To enable integration with a variety of enterprise databases, DataRobot provides a “self-service” JDBC product for database connectivity setup. Once configured, you can read data from production databases for model building and predictions. This allows you to quickly train and retrain models on that data, and avoids the unnecessary step of exporting data from your enterprise database to a CSV for ingest to DataRobot. It allows access to more diverse data, which results in more accurate models.

The steps describing how to set up your database connections use the following terminology:

- **DataStore**: A configured connection to a database&mdash; it has a name, a specified driver, and a JDBC URL. You can register data stores with DataRobot for ease of re-use. A data store has one connector but can have many data sources.
- **DataSource**: A configured connection to the backing data store (the location of data within a given endpoint). A data source specifies, via SQL query or selected table and schema data, which data to extract from the data store to use for modeling or predictions. A data source has one data store and one connector but can have many datasets.
- **DataDriver**: The software that allows the DataRobot application to interact with a database; each data store is associated with one driver (created by the admin). The driver configuration saves the storage location in DataRobot of the JAR file and any additional dependency files associated with the driver.
- **Dataset**: Data, a file or the content of a data source, at a particular point in time. A data source can produce multiple datasets; a dataset has exactly one data source. When a DataSource, file, url, or local dataframe is instantiated as a `Dataset`, it shows up in the **AI Catalog**. 

**NOTE**: With Python `datarobot` v2.21.3, an AI Catalog `Dataset` can only be created from URL, in-memory dataframes (pandas), and local files. In Python `datarobot` v2.22, you will be able to create Datasets from DataSources (e.g. Database tables). You can however, still leverage DataSources to create projects, the snapshotted dataset just won't show up in the Catalog. Through the GUI you can currently instantiate an AI Catalog `Dataset` from a Database table

### Pre-conditions for this notebook
- `diabetes_readmission.csv` and `diabetes_readmission_predict.csv` are loaded into database of your choosing, make sure the table names match and/or change the queries in the below code appropriately
- Your installation of DataRobot can access that database

In [2]:
import datarobot as dr
import pandas as pd
from datarobot.models.modeljob import wait_for_async_model_creation
from datarobot.enums import AVAILABLE_STATEMENT_TYPES
from io import StringIO
import requests

# Database credentials - CHANGE THESE
USERNAME = ''
PASSWORD = ''
BLUEPRINT_TRAIN = 'eXtreme Gradient Boosted Trees Classifier with Early Stopping'
JDBC_URL = ''

# DataRobot credentials - CHANGE THESE
API_TOKEN = ''
DATAROBOT_KEY = '' # don't need the key if on-prem
ENDPOINT = 'https://app.datarobot.com/api/v2' 

client = dr.Client(token=API_TOKEN, endpoint=ENDPOINT)

In [3]:
drivers = dr.DataDriver.list()
drivers

[DataDriver('Microsoft SQL Server (6.0)'),
 DataDriver('Redshift (1.2.12)'),
 DataDriver('Snowflake (3.5.4)'),
 DataDriver('PostgreSQL (9.4.1208)'),
 DataDriver('Microsoft SQL Server (6.4)'),
 DataDriver('Elasticsearch (6.4.0)'),
 DataDriver('Redshift (1.2.34)'),
 DataDriver('AWS Athena 2.0 (2.0.5)'),
 DataDriver('MySQL (5.1.44)'),
 DataDriver('Treasure Data (0.5.10)'),
 DataDriver('Presto (0.216)'),
 DataDriver('Redshift (1.2.34)'),
 DataDriver('Microsoft SQL Server (6.4-jre1.8)'),
 DataDriver('Apache Hive for JDBC 4.1'),
 DataDriver('Oracle6'),
 DataDriver('Oracle_8'),
 DataDriver('Apache Hive 1.1'),
 DataDriver('kdb+ (2019.11.11)'),
 DataDriver('Microsoft SQL Server (v7.4.1)'),
 DataDriver('MySQL (5.1.24)'),
 DataDriver('InterSystems'),
 DataDriver('Snowflake (3.12.0)'),
 DataDriver('Google BigQuery (1.2.2)'),
 DataDriver('PostgreSQL (42.0.0)'),
 DataDriver('SAP HANA (2.4.70)'),
 DataDriver('Google BigQuery (spark-2.4.5)')]

In [4]:
# Change Driver according to your DB
mssql_server_driver = [d for d in dr.DataDriver.list() if d.canonical_name == 'Microsoft SQL Server (v7.4.1)'][0]

#### Create a DataStore for a MS SQL Server DB

In [5]:
mssql_server_datastore = dr.DataStore.create(data_store_type='jdbc',
                                             canonical_name='DataRobot API Training DB',
                                             driver_id=mssql_server_driver.id,
                                             jdbc_url=JDBC_URL)

In [6]:
mssql_server_datastore = [x for x in dr.DataStore.list() if x.canonical_name == 'DataRobot API Training DB'][0]
mssql_server_datastore.tables(username=USERNAME, password=PASSWORD)

{'tables': [{'catalog': 'humana_demo_db',
   'type': 'TABLE',
   'name': 'diabetes_readmission',
   'schema': 'dbo'},
  {'catalog': 'humana_demo_db',
   'type': 'TABLE',
   'name': 'diabetes_readmission_predict',
   'schema': 'dbo'},
  {'catalog': 'humana_demo_db',
   'type': 'TABLE',
   'name': 'scored_patients',
   'schema': 'dbo'},
  {'catalog': 'humana_demo_db',
   'type': 'TABLE',
   'name': 'trace_xe_action_map',
   'schema': 'sys'},
  {'catalog': 'humana_demo_db',
   'type': 'TABLE',
   'name': 'trace_xe_event_map',
   'schema': 'sys'},
  {'catalog': 'humana_demo_db',
   'type': 'VIEW',
   'name': 'CHECK_CONSTRAINTS',
   'schema': 'INFORMATION_SCHEMA'},
  {'catalog': 'humana_demo_db',
   'type': 'VIEW',
   'name': 'COLUMN_DOMAIN_USAGE',
   'schema': 'INFORMATION_SCHEMA'},
  {'catalog': 'humana_demo_db',
   'type': 'VIEW',
   'name': 'COLUMN_PRIVILEGES',
   'schema': 'INFORMATION_SCHEMA'},
  {'catalog': 'humana_demo_db',
   'type': 'VIEW',
   'name': 'COLUMNS',
   'schema': 'INFO

#### Create a DataSource from the `diabetes_readmission` table

In [7]:
params = dr.DataSourceParameters(data_store_id=mssql_server_datastore.id,
                                 query='SELECT * from diabetes_readmission;')
diabetes_data_source = dr.DataSource.create(data_source_type='jdbc',
                                            canonical_name='dr_api_training_diabetes_readmission',
                                            params=params)

#### Create a Project from the DataSource

In [8]:
proj = dr.Project.create_from_data_source(data_source_id=diabetes_data_source.id,
                                          username=USERNAME,
                                          password=PASSWORD)
proj.set_target(target="readmitted",
                mode=dr.AUTOPILOT_MODE.MANUAL,
                worker_count=-1)
dr_xgboost = [b for b in proj.get_blueprints() if BLUEPRINT_TRAIN in b.model_type][0]
model_job_id = proj.train(dr_xgboost, sample_pct=16)
dr_xgboost_model = wait_for_async_model_creation(
    project_id=proj.id,
    model_job_id=model_job_id,
)
dr_xgboost_model_id = dr_xgboost_model.id

## <b>Model Deployment with the API<b>

<b>Prediction Servers<b>
- Prediction Servers are dedicated servers where you can deploy DataRobot models. Models deployed here are exposed with a REST API you can use to send new data for preditions 
- Or you can use the `BatchPredictionJob` class to kick off large-scale asynchronous scoring requests to the DataRobot models living on these servers.
    

In [9]:
dr.PredictionServer.list()
prediction_server = dr.PredictionServer.list()[0]

In [10]:
deployment = dr.Deployment.create_from_learning_model(dr_xgboost_model.id, 
                                                      label=f'test_{dr_xgboost_model.id}_deployment', 
                                                      description='diabetes demo',
                                                      default_prediction_server_id=prediction_server.id)

# Update the deployment to track target and feature drift
deployment.update_drift_tracking_settings(target_drift_enabled=True, 
                                          feature_drift_enabled=True, 
                                          max_wait=600)

<b> Once deployed to the server, you can send new data to the model for predictions in two ways: <b>
    
| API Option  | How It Works | Caveats |
| ----------- | -----------  | ----------- |
| REST Prediction API | Send a HTTP `post` request with data to be scored encoded as json or csv | - synchronous (the HTTP response contains the result), <br >  - 50 MB Limit per request <br > - low latency|
| Python Batch Prediction API   |   Starts a scoring job that automatically streams data directly to and from files or databases prediction servers for prediction. This utility is part of the `datarobot` package  | - asynchronous (must explicitly wait for completion or check status) <br > - unlimited data limit per job <br > - slower than the REST API|
    
<b> REST Prediction API <b>
- You will find a code example of this in every deployment you create in DataRobot under "Integrations" -> "Scoring Code"
- Works like a standard web request and you can choose to encode your data as json or csv
- Best for high frequency, small payloads workloads e.g. information coming in real-time from a UI or website
- You can use packages like `request_futures` and HTTP concurrency to score larger datasets in 50 MB batches concurrently

In [11]:
# Specify ID of deployment and Prediction API endpoint
headers = {
        'Content-Type': 'text/csv;charset=utf8',
        'Authorization': f'Bearer {API_TOKEN}',
        'DataRobot-Key': DATAROBOT_KEY,    # Not needed for on-prem
        "Accept": "text/csv"
    }

url = f'{prediction_server.url}/predApi/v1.0/deployments/{deployment.id}/predictions'


prediction_batch = pd.read_csv('diabetes_readmission_predict.csv')

# Make API request for predictions
predictions_response = requests.post(
    url,
    data=prediction_batch.to_csv(),
    headers=headers
)
result_df = pd.read_csv(StringIO(predictions_response.text))

In [12]:
result_df

,readmitted_1_PREDICTION,readmitted_0_PREDICTION,readmitted_PREDICTION,THRESHOLD,POSITIVE_CLASS
0,0.432907,0.567093,0,0.5,1
1,0.265067,0.734933,0,0.5,1
2,0.493953,0.506047,0,0.5,1
3,0.448824,0.551176,0,0.5,1
4,0.381380,0.618620,0,0.5,1
...,...,...,...,...,...
95,0.576642,0.423358,1,0.5,1
96,0.285731,0.714269,0,0.5,1
97,0.319893,0.680107,0,0.5,1
98,0.457613,0.542387,0,0.5,1


<b> Batch Prediction API <b>
- **Unlimited** scoring limit (except for s3)
- Score from file -> db, db-> file, or file->file
- Keep in mind these are asynchronous and you have to manually block or check status of these jobs
- Best for low-frequency, large payloads

In [13]:
db_credentials = dr.models.Credential.create_basic(name='demo_credentials',
                                                   user=USERNAME,
                                                   password=PASSWORD,
                                                   description='credentials for API Training Database')

In [14]:
db_credentials.list()

[Credential('5e1523774322bb048f5e5597', 'kenlee', 'basic'),
 Credential('5ecaff8af3ab9853c7f96c25', 'demo', 'basic'),
 Credential('5f2041260aa8f11cd4ea55bc', 'datarobot', 'basic'),
 Credential('5f21ce290aa8f12bc5ea5736', 'SourceCredentials_PredicitonJob_5f21ce280aa8f12bc5ea5733', 'basic'),
 Credential('5f21ce290aa8f12bc5ea5738', 'TargetCredentials_PredicitonJob_5f21ce280aa8f12bc5ea5733', 'basic'),
 Credential('5f3353979071ae3d12d6df02', 'github-application-oauth', 'oauth'),
 Credential('5f3d82632c619c1dc1d2bec0', 'demo_credentials', 'basic')]

In [19]:
# Note you have to copy the credential id below
db_credentials_id = '5f3d82632c619c1dc1d2bec0'

In [21]:
# Unscored data in database table -> scored data in new database table

job = dr.BatchPredictionJob.score(
    deployment.id,
    passthrough_columns_set='all',
    max_explanations=3,
    intake_settings = {
        'type': 'jdbc',
        'query': 'select * from diabetes_readmission_predict',
        'data_store_id': mssql_server_datastore.id,
        'credential_id': db_credentials_id,
    },
    output_settings = {
        'type': 'jdbc',
        'table': 'scored_patients',
        'schema': 'dbo',
        'statement_type': AVAILABLE_STATEMENT_TYPES.CREATE_TABLE,
        'data_store_id': mssql_server_datastore.id,
        'credential_id': db_credentials_id,
    },
)

job.wait_for_completion()

In [22]:
# Unscored data in local file -> scored data in local file

prediction_server = dr.PredictionServer.list()[0]
job = dr.BatchPredictionJob.score(deployment=deployment.id,
                                  passthrough_columns_set='all',
                                  intake_settings={
                                        'type': 'localFile',
                                        'file': 'diabetes_readmission_predict.csv'
                                  },
                                  output_settings={
                                         'type': 'localFile',
                                         'path': 'scored_patients.csv'
                                  })
job.wait_for_completion()

In [23]:
# Delete a deployent
deployment.delete()
proj.delete()
diabetes_data_source.delete()
mssql_server_datastore.delete()
db_credentials.delete()